# Project task 05:  Clustering users based on their preferences

In [1]:
import numpy as np
import scipy.sparse as sp

The goal of this task is to find groups of users with similar preferences using **Spectral clustering**. 
You are given a fragment of the Yelp social network, represented by an undirected weighted graph.
Nodes in the graph represent users.
If two users are connected by an edge of weight $w$, it means that they have both left positive reviews to the same $w$ restaurants.

Additionally, you are given a matrix `F` that encodes user preferences to different categories of restaurants. If `F[i, c] = 1`, then user `i` likes restaurants in category `c`.

You are allowed to use the imported functions (`eigsh`, `KMeans`, `normalize`).

## Load the data

* `N` = number of users (nodes in the graph)
* `C` = number of categories
* The graph is stored as a sparse adjacency matrix `A` (shape `[N, N]`).
* User preferences are stored in a matrix `F` (shape `[N, C]`). They will only be used for the final part of the assignment (Part 3)
* Name of each category is provided in the list `categories` (length `[C]`).

In [2]:
A = sp.load_npz('A.npz')
F = np.load('F.npy')
categories = np.load('categories.npy').tolist()

In [3]:
assert A.shape[0] == F.shape[0]
assert F.shape[1] == len(categories)

In [153]:
A.sum()

4359968.0

In [122]:
np.sum(A,axis = 1)

matrix([[2255.],
        [3462.],
        [1122.],
        ...,
        [1344.],
        [ 528.],
        [1368.]])

In [9]:
F

array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 1., 0., 0.],
       [0., 1., 1., ..., 0., 0., 1.]])

In [10]:
categories

['African',
 'American (New)',
 'American (Traditional)',
 'Arcades',
 'Arts & Crafts',
 'Arts & Entertainment',
 'Asian Fusion',
 'Bagels',
 'Bakeries',
 'Barbeque',
 'Beer',
 'Beer Bar',
 'Belgian',
 'Bistros',
 'Brasseries',
 'Brazilian',
 'Breakfast & Brunch',
 'Breweries',
 'British',
 'Bubble Tea',
 'Buffets',
 'Burgers',
 'Cafes',
 'Cajun/Creole',
 'Caribbean',
 'Caterers',
 'Chicken Shop',
 'Chicken Wings',
 'Chinese',
 'Chocolatiers & Shops',
 'Cocktail Bars',
 'Coffee & Tea',
 'Comfort Food',
 'Creperies',
 'Cuban',
 'Dance Clubs',
 'Delicatessen',
 'Delis',
 'Desserts',
 'Dim Sum',
 'Diners',
 'Dive Bars',
 'Do-It-Yourself Food',
 'Ethiopian',
 'Ethnic Food',
 'Event Planning & Services',
 'Falafel',
 'Fast Food',
 'Filipino',
 'Fish & Chips',
 'Food Court',
 'Food Delivery Services',
 'Food Stands',
 'Food Trucks',
 'French',
 'Gastropubs',
 'German',
 'Gluten-Free',
 'Greek',
 'Grocery',
 'Halal',
 'Hawaiian',
 'Health Markets',
 'Himalayan/Nepalese',
 'Hobby Shops',
 'Hot

# 1. Implementing spectral clustering
## 1.1. Construct the graph Laplacian

First, we need to construct the Laplacian for the given graph. 

Given the **adjacency matrix** $A \in \mathbb{R}^{N \times N},$ we define the **degree matrix** $D \in \mathbb{R}^{N \times N}$ of an undirected graph as
$$D_{ij} =  \begin{cases}\sum_{k=1}^N A_{ik} & if \;\; i = j\\ 0 & if \;\; i \ne j\end{cases}$$

If our goal is to minimize the **ratio cut**, we will need to use the **unnormalized Laplacian**, defined as
$$L_{unnorm} = D - A.$$

If our goal is to minimze the **normalized cut**, we will need to use the **normalized Laplacian** (a.k.a. symmetrized Laplacian), defined as
$$L_{sym} = I - D^{-1/2}AD^{-1/2}$$

In [137]:
def construct_laplacian(A, norm_laplacian):
    """Construct Laplacian of a graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    norm_laplacian : bool
        Whether to construct the normalized graph Laplacian or not.
        If True, construct the normalized (symmetrized) Laplacian, L = I - D^{-1/2} A D^{-1/2}.
        If False, construct the unnormalized Laplacian, L = D - A.
        
    Returns
    -------
    L : scipy.sparse.csr_matrix, shape [N, N]
        Laplacian of the graph.
        
    """
    ### YOUR CODE HERE ###
    from scipy.sparse import csgraph
    N = A.shape[0]
    D = np.zeros((N,N))
#     I = np.ones((N,N))
    
    row,col =np.diag_indices_from(D)
    D[row,col] = np.array(np.sum(A,axis = 1)).ravel()
#     print(D)
    L = D - A
#     print(L)
#     print(D[row,col].shape)
#     print(np.array(np.sum(A,axis = 1)).ravel().shape)

    if(norm_laplacian):
        D[row,col]=np.power(D[row,col],-1/2)
#         print(D)
        L = (D.dot(L)).dot(D)
#         L2 = I - np.dot(np.dot(D,A),D)
#         print(L==L2)

#         print(csgraph.laplacian(L, normed=True))

    return L

In [138]:
construct_laplacian(A,True)

matrix([[ 1.00000000e+00, -3.57901233e-04, -6.28680947e-04, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-3.57901233e-04,  1.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00, -1.47927699e-03,  0.00000000e+00],
        [-6.28680947e-04,  0.00000000e+00,  1.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          1.00000000e+00, -1.18708934e-03, -1.47498401e-03],
        [ 0.00000000e+00, -1.47927699e-03,  0.00000000e+00, ...,
         -1.18708934e-03,  1.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         -1.47498401e-03,  0.00000000e+00,  1.00000000e+00]])

## 1.2. Spectral embedding

Now, we have to compute the spectral embedding for the given graph.

In order to partition the graph into $k$ clusters, such that the desired cut (ratio or normalized) is minimized, we need to consider the $k$ eigenvectors corresponding to the $k$ smallest eigenvalues of the graph Laplacian.

Since the Laplacian matrix is sparse and symmetric, we can use the function `eigsh` from the `scipy.sparse.linalg` package in order to find eigendecomposition of $L$ (`eig` - eigendecomposition, `s` - sparse, `h`- Hermitian).
The function `eigsh` directly allows you to find the smallest / largest eigenvalues by specifying the `k` and `which` parameters. 

Keep in mind that the Laplacian matrix is always positive semi-definite when picking the appropriate value for the `which` parameter.

In [5]:
from scipy.sparse.linalg import eigsh

In [169]:
def spectral_embedding(A, num_clusters, norm_laplacian):
    """Compute spectral embedding of nodes in the given graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    num_clusters : int
        Number of clusters to detect in the data.
    norm_laplacian : bool, default False
        Whether to use the normalized graph Laplacian or not.
        
    Returns
    -------
    embedding : np.array, shape [N, num_clusters]
        Spectral embedding for the given graph.
        Each row represents the spectral embedding of a given node.
    
    """
    if (A != A.T).sum() != 0:
        raise ValueError("Spectral embedding doesn't work if the adjacency matrix is not symmetric.")
    ### YOUR CODE HERE ###
    L = construct_laplacian(A,norm_laplacian)
    eigenvalues, eigvecs = eigsh(L, k=num_clusters, which='SA')
#     print(eigenvalues)
    
    return eigvecs

In [149]:
spectral_embedding(A,6,False)

[-2.19710608e-12  6.79046066e+01  6.85357173e+01  7.22045628e+01
  7.79112271e+01  8.17477831e+01]


array([[-0.01719035,  0.00027954,  0.00039815,  0.00020549,  0.00029841,
        -0.00033078],
       [-0.01719035,  0.00027944,  0.00039683,  0.00018978,  0.00028385,
        -0.00032446],
       [-0.01719035,  0.00026292,  0.00043   ,  0.00020734,  0.00031677,
        -0.00034699],
       ...,
       [-0.01719035,  0.00027733,  0.00042267,  0.00020251,  0.00030726,
        -0.00033546],
       [-0.01719035,  0.0002976 ,  0.00044688,  0.000234  ,  0.00034747,
        -0.0003704 ],
       [-0.01719035,  0.00028916,  0.00039692,  0.0001946 ,  0.00029752,
        -0.00034005]])

## 1.3. Determine the clusters based on the spectral embedding

You should use the K-means algorithm for assigning nodes to clusters, once the spectral embedding is computed.

One thing you should keep in mind, is that when using the **normalized Laplacian**, the rows of the embedding matrix **have to** be normalized to have unit $L_2$ norm.

In [7]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize

In [140]:
def spectral_clustering(A, num_clusters, norm_laplacian):
    """Perform spectral clustering on the given graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    num_clusters : int
        Number of clusters to detect in the data.
    norm_laplacian : bool, default False
        Whether to use the normalized graph Laplacian or not.
        
    Returns
    -------
    z_pred : np.array, shape [N]
        Predicted cluster indicators for each node.
        
    """
    ### YOUR CODE HERE ###
    eigvecs = spectral_embedding(A, num_clusters, norm_laplacian)
    if(norm_laplacian):
        eigvecs = normalize(eigvecs, norm='l2')
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(eigvecs)
    z_pred = kmeans.labels_
        
    
    return z_pred

In [145]:
spectral_clustering(A,6,False)

array([0, 0, 0, ..., 0, 0, 0])

# 2. Quantitatively evaluate the results

In [62]:
def labels_to_list_of_clusters(z):
    """Convert predicted label vector to a list of clusters in the graph.
    This function is already implemented, nothing to do here.
    
    Parameters
    ----------
    z : np.array, shape [N]
        Predicted labels.
        
    Returns
    -------
    list_of_clusters : list of lists
        Each list contains ids of nodes that belong to the same cluster.
        Each node may appear in one and only one partition.
    
    Examples
    --------
    >>> z = np.array([0, 0, 1, 1, 0])
    >>> labels_to_list_of_clusters(z)
    [[0, 1, 4], [2, 3]]
    
    """
    return [np.where(z == c)[0] for c in np.unique(z)]

In [158]:
len(labels_to_list_of_clusters(spectral_clustering(A,6,False))[0])

[-3.64684441e-13  6.79046066e+01  6.85357173e+01  7.22045628e+01
  7.79112271e+01  8.17477831e+01]


3379

## 2.1. Compute ratio cut

Your task is to implement functions for computing the **ratio cut** and **normalized cut** for a given partition.

Ratio cut and normalized cut are defined on the slide 14 of the lecture slides.


The function `labels_to_list_of_clusters` can be helpful here.

In [171]:
def compute_ratio_cut(A, z):
    
    """Compute the ratio cut for the given partition of the graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    z : np.array, shape [N]
        Cluster indicators for each node.
    
    Returns
    -------
    ratio_cut : float
        Value of the cut for the given partition of the graph.
        
    """
    
    ### YOUR CODE HERE ###
    from itertools import combinations
    
    clusters = labels_to_list_of_clusters(z)
#     print('clusters',clusters)

    numSet = set(range(A.shape[0]))
    result=[]
    
    for i in range(len(clusters)):

        s = list(clusters[i])
        length = len(s)
        
        column = list(numSet - set(s))
        
        A2 = A[s]
        A2 = A2[:,column]
        cut2 = A2.sum()
        
        result.append(cut2/length)

    ratio_cut = sum(result)


    return ratio_cut

## 2.2. Compute normalized cut

**Important**: if a cluster only contains a single node, define its volume to be 1 to avoid division by zero errors.

In [174]:
def compute_normalized_cut(A, z):
    """Compute the normalized cut for the given partition of the graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    z : np.array, shape [N]
        Cluster indicators for each node.
    
    Returns
    -------
    norm_cut : float
        Value of the normalized cut for the given partition of the graph.
        
    """
    
    ### YOUR CODE HERE ###
    
    from itertools import combinations
    
    clusters = labels_to_list_of_clusters(z)
    numSet = set(range(A.shape[0]))
    result=[]

    for i in range(len(clusters)):
        #print(i)
        s = list(clusters[i])
        vol = A[s,:].sum()
        
        column = list(numSet - set(s))
        
        A2 = A[s]
        A2 = A2[:,column]
        cut2 = A2.sum()
        
        result.append(cut2/vol)
        
        
    norm_cut = sum(result)
    
    return norm_cut

Notice, how using the unnormalized Laplacian leads to a much better ratio cut, while the normalized Laplacian leads to better normalized cut.

In [65]:
num_clusters = 6

In [176]:
norm_laplacian = False
z_unnorm = spectral_clustering(A, num_clusters, norm_laplacian)
print(z_unnorm)
print('When using L_unnorm:')
print(' ratio cut = {:.3f}'.format(compute_ratio_cut(A, z_unnorm)))
print(' normalized cut = {:.3f}'.format(compute_normalized_cut(A, z_unnorm)))
print(' sizes of partitions are: {}'.format([len(clust) for clust in labels_to_list_of_clusters(z_unnorm)]))

[0 0 0 ... 0 0 0]
When using L_unnorm:
 ratio cut = 369.109
 normalized cut = 5.000
 sizes of partitions are: [3379, 1, 1, 1, 1, 1]


In [175]:
norm_laplacian = True
z_norm = spectral_clustering(A, num_clusters, norm_laplacian)
print('When using L_norm:')
print(' ratio cut = {:.3f}'.format(compute_ratio_cut(A, z_norm)))
print(' normalized cut = {:.3f}'.format(compute_normalized_cut(A, z_norm)))
print(' sizes of partitions are: {}'.format([len(clust) for clust in labels_to_list_of_clusters(z_norm)]))

When using L_norm:
 ratio cut = 5942.851
 normalized cut = 4.343
 sizes of partitions are: [742, 754, 350, 389, 577, 572]


# 3. Visualize the results

In the final part of the assignment, your task is to print out the 5 most popular types of restaurants visited by the users in each cluster.

In [179]:
def print_top_categories_for_each_cluster(top_k, z, F, categories):
    """Print the top-K categories among users in each cluster.
    For each cluster, the function prints names of the top-K categories,
    and number of users that like the respective category (separated by a comma).
    The function doesn't return anything, just prints the output.
    
    Parameters
    ----------
    top_k : int
        Number of most popular categories to print for each cluster.
    z : np.array, shape [N]
        Cluster labels.
    F : sp.csr_matrix, shape [N, C]
        Matrix that tells preferences of each user to each category.
        F[i, c] = 1 if user i gave at least one positive review to at least one restaurant in category c.
    categories : list, shape [C]
        Names of the categories.
        
    """
    ### YOUR CODE HERE ###
    clusters = labels_to_list_of_clusters(z)
    for i in range(len(clusters)):
        s = list(clusters[i])
        F2 = F[s]
        sumLike = np.sum(F2,axis = 0)
        rank = sumLike.argsort()[-top_k:][::-1]
        
        print('Most popular categories in cluster', i)
        for i in range(top_k):
            print(categories[rank[i]],sumLike[rank[i]])
        
        print()
        
        
        

In [181]:
np.random.seed(2)
z_norm = spectral_clustering(A, num_clusters, True)
r = print_top_categories_for_each_cluster(5, z_norm, F, categories)

Most popular categories in cluster 0
Breakfast & Brunch 636.0
Sandwiches 528.0
Italian 514.0
Pizza 482.0
Coffee & Tea 473.0

Most popular categories in cluster 1
Breakfast & Brunch 664.0
Italian 626.0
American (Traditional) 518.0
Sandwiches 518.0
Pizza 485.0

Most popular categories in cluster 2
Seafood 315.0
Mexican 314.0
Sandwiches 294.0
Japanese 291.0
Breakfast & Brunch 286.0

Most popular categories in cluster 3
Specialty Food 356.0
Thai 355.0
Breakfast & Brunch 348.0
Japanese 333.0
Ethnic Food 330.0

Most popular categories in cluster 4
Japanese 529.0
Chinese 441.0
Asian Fusion 414.0
Sushi Bars 408.0
Desserts 406.0

Most popular categories in cluster 5
Japanese 507.0
Breakfast & Brunch 462.0
Sandwiches 435.0
Italian 417.0
Asian Fusion 414.0

